<div style="text-align: center;">
        <img src="images/molssi_logo_header.png" alt="MolSSI log" style="height: 250px;">
</div>

# Docking Preparation

<div class="alert alert-block alert-info"> 
<h2>Overview</h2>
    
<strong>Questions</strong>

* How do I prepare structure files for docking?

<strong>Learning Objectives</strong>

* Use the RCSB Search API to find a protein structure for docking.
* Prepare a protein and ligand structures for input into AutoDock Vina.
</div>

In this notebook, we will get our structure files ready for docking.
We will need to create a special file format called pdbqt, which is used by AutoDock Vina.
The PDBQT format is similar to the PDB format, but it includes additional information such as atomic charges.

We will need to complete a few steps:

1. Download a trypsin protein structure with our ligand of interest docked from the PDB.
2. Isolate the protein in the structure (strip any water molecules and ligand).
3. Add hydrogens to the protein and clean up the structure.
4. Create a PDBQT file for our protein.
5. Create PDBQT files for our ligands.
### Python Libraries for the IQB workshop

We'll be using many of the same libraries that we used in previous notebooks.
However, this time we will add one more library to our list - [MDAnalysis](https://www.mdanalysis.org/).
MDAnalysis is a Python library that was written for analyzing molecular dynamics (MD) simulations.
However, it has reading and writing capabilities for many molecular file formats, as well as a selection language for isolating particular parts of molecules.

| Library         | abbreviation | Purpose |
|:-------------|:---------:|:------------|
| rcsbsearchapi | N/A      | functions for searching the Protein Data Bank based on the mmCIF dictionary |
| requests     | N/A  | access web URLs - used with APIs for databases |
| os           | N/A      | operating system functions - handling file paths and directories. |
| nglview      | nv       | for viewing molecular structures |
| MDAnalysis     | mda | molecular dynamics library - used for reading/writing files and selecting atoms |

### Command Line Software Used in this Notebook

We'll also be adding a few command line scripts and utilities to this notebook.
Usually, these would be executed in the terminal, but we'll run them from the Jupyter interface.
These will be used to prepare our structures for docking calculations

| Software         | Purpose |
|:-------------|:---------|
| pdb2pqr      | adding hydrogens and missing atoms to protein, adjusting for pH |
| meeko        | preparing ligands for docking |

## Finding and saving a protein structure

To find an appropriate protein for our docking study, we can again use the PDB Search API, similar to how we used it in the first notebook.
This time, we will use the same EC number, but the second part of our query will be our ligand name.


In [ ]:
from rcsbsearchapi.search import TextQuery
from rcsbsearchapi import rcsb_attributes as attrs

ECnumber = "3.4.21.4"    

q1 = attrs.rcsb_polymer_entity.rcsb_ec_lineage.id == ECnumber  # looking for trypsins
q2 = TextQuery("13U")

query = q1 & q2               # combining the two queries into one

results = list(query())
print(results)

In [ ]:
pdb_id = results[0].lower() # Get the PDB ID and convert to lowercase
print(pdb_id)

We have identified the PDB ID of a trypsin with our ligand of interest bound. 
Our PDB ID is `2zq2`, and we saved it in a variable in the cell above.

We will now create a directory called `protein_structures` to save this in.

In [ ]:
import os # for making directories
import requests

# make a directory for pdb files
protein_directory = "protein_structures"
os.makedirs(protein_directory, exist_ok=True)

pdb_request = requests.get(f"https://files.rcsb.org/download/{pdb_id}.pdb")
pdb_request.status_code

In [ ]:
with open(f"{protein_directory}/{pdb_id}.pdb", "w+") as f:
    f.write(pdb_request.text)

## Visualizing the protein strucure

Before we start to really work with our molecule, let's investigate the structure.
We will use a library called MDAnalysis to first process our PDB, then visualize it with a library called NGLView.

MDAnalysis is a Python library that is used to process molecular dynamics trajectories and other 3D strucure molecular files.
The core object for MDAnalysis is a "Universe" and it corresponds to a molecular system.
We can load a PDB file into MDAnalysis, then do things like measure distances in our structure or isolate particular parts.

In [ ]:
import MDAnalysis as mda

# Load into MDA universe
u = mda.Universe(f"{protein_directory}/{pdb_id}.pdb")
u

After loading our PDB, we can see that we have an MDAnalysis "universe" (or molecular system) that contains 2024 atoms. 
We can inspect this structure visually using a library called NGLView.
NGLView is a molecular visualizer made to work on the web and Jupyter notebooks.
If you're interested in learning about NGLView, you can see a [video from MolSSI's first Crash Course with the PDB](https://www.youtube.com/watch?v=6QHWhycMuXc). 

In [ ]:
import nglview as nv
view = nv.show_mdanalysis(u)
view

This view looks a bit messy, and we likely want to isolate the protein and ligand for viewing.
MDAnalysis has a human readable [selection syntax](https://docs.mdanalysis.org/stable/documentation_pages/selections.html)
that allows us to isolate parts of our structure. We will take our MDAnalysis Universe (the variable `u`) and use the `select_atoms` function.
Inside this function, we will fill in what we want to select.

We will create separate variables for the protein and ligand. We can select all protein residues in MDAnalysis using the word "protein" in the `select_atoms` function. Then, we will select our ligand using `resname 13U`. This corresponds to the residue name in the PDB we downloaded.
We can also select waters in the structure by using `"resname HOH"`.

In [ ]:
# Select protein atoms
protein = u.select_atoms("protein")
ligand = u.select_atoms("resname 13U")
water = u.select_atoms("resname HOH")

water

After selecting parts of our structure, we can add them individually to an NGLView view.
In the cell below, we visualize the protein's surface area colored by hydrophobicity.
Waters from the crystal structure are in spacefill representation, and we add the ligand in a ball and stick representation.

In [ ]:
view = nv.show_mdanalysis(protein)
view.clear_representations()
view.add_representation("surface", colorScheme="hydrophobicity")
lig_view = view.add_component(ligand)
lig_view.add_representation("ball+stick")
water_view = view.add_component(water)
water_view.add_representation("spacefill")
view

If you rotate this structure so that you are looking at the bottom, you will be able to see our `13U` ligand bound.
Upon viewing this structure, you will notice that our ligand seems to appear twice. 
If you open the PDB file to investigate, you will see the following in the ligand section:

```
HETATM 1673  C14A13U A 501      18.144  -9.216  12.088  0.61 24.22           C  
ANISOU 1673  C14A13U A 501     1755   4793   2654   1752    148   1233       C  
HETATM 1674  C14B13U A 501      18.147  -8.840  11.672  0.39 24.46           C  
ANISOU 1674  C14B13U A 501     2583   4283   2430   1765    353   1279       C  
HETATM 1675  O32A13U A 501      18.209  -8.355  11.186  0.61 24.38           O  
ANISOU 1675  O32A13U A 501     2354   5394   1514   2217    238    919       O
```

This PDB structure provides [**alternate locations**](https://proteopedia.org/wiki/index.php/Alternate_locations) for each ligand atom. 
These occur when the experimental data supports multiple positions for the same atom.
In excerpt above, you will see C14A13U and C14B13U. These are alternate locations of the same atom. 
Alternate locations can also occur in the protein with some residues.

<div class="alert alert-block alert-success">
<strong>Selecting alternate locations using MDAnalysis</strong>  
    
By checking the [documentation page](https://docs.mdanalysis.org/stable/documentation_pages/selections.html) for MDAnalysis selections, we can see that MDAnalysis is prepared for this scenario. We will want to use the `altloc` keyword. This keyword is described as:

> altLoc alternative-location

> a selection for atoms where alternative locations are available, which is often the case with high-resolution crystal structures e.g. resid 4 and resname ALA and altLoc B selects only the atoms of ALA-4 that have an altLoc B record.

If you wanted to use MDAnalysis to select for a particular ligand location, you could use:

```python
ligand_A = u.select_atoms("resname 13U and altLoc A")
ligand_B = u.select_atoms("resname 13U and altLoc B")
```
</div>

To perform a docking calculation, we will have to isolate the protein.
This starting structure for our protein contains extra molecules like ligands and water.
You will also notice from examining our visualization that our structure does not include hydrogen atoms.
If you were to examine the PDB file, you would also see that there are some missing atoms and some of our protein residues have alternate locations marked just like thie ligand.

For docking, we will want to remove all of these extra molecules and only keep the protein.
We will also want to remove any alternate locations of residues.
We will use MDAnalysis to remove these extra molecules and save our starting protein structure as a new file.

In [ ]:
# Write protein to new PDB file
protein.write(f"{protein_directory}/protein_{pdb_id}.pdb")

<div class="alert alert-block alert-danger">
<strong>Protein Charge</strong>  

After saving the protein in the cell above, you may see a warning about information for formal charges not being set in the protein. 
This warning appears because MDAnalysis did not find specific formal charge data in the PDB file and used a default value instead. 
This is not a concern for us because we will adjust the protonation states of different residues using PDB2PQR in the next steps. 
</div>


## Fixing the protein structure

Now that we've isolated our protein, we will want to ensure that we've correctly added hydrogen and fixed any missing atoms.

For fixing our protein, we will use a specialized program called PDB2PQR that is made for working with biomolecules like proteins.
The advantage of using PDB2PQR is that it will check our protein for missing atoms and multiple occupancy in the protein, and it will pick positions and add missing atoms.

<div class="alert alert-block alert-success">
<strong>More complicated fixes: PDBFixer</strong>  

Another popular software for fixing PDB files is called [PDBFixer](https://github.com/openmm/pdbfixer). PDBFixer is an open-source tool developed by the OpenMM team and is designed to fix common problems in Protein Data Bank (PDB) files before they are used in molecular simulations. It can be used to remove unwanted molecules like water, add missing heavy atoms to incomplete residues, add hydrogen atoms where needed.

PDBFixer can be especially useful when there are missing loops or residues. In this workshop, our protein is not missing any residues, but many proteins from the PDB might require more adjustment.

To see an example of preparing proteins with PDBFixer, see this [recent YouTube video](https://www.youtube.com/watch?v=pwfKE6wPaMg) posted by the Open Forcefield Initiative. In this video, the presenter first uses PDBFixer, then PDB2PQR to adjust protonation.

</div>

We will use the command-line interface of this PDB2PQR. This means that you would usually type the command below into your terminal
You can run command line commands in the Jupyter notebook by putting a `!` in front of the command. 

In [ ]:
! pdb2pqr --pdb-output=protein_structures/protein_h.pdb --pH=7.4 protein_structures/protein_2zq2.pdb protein_structures/protein_2zq2.pqr --whitespace

## Saving a protein PDBQT File

The PDB2PQR program outputs two files, a PDB file and a PQR file. The PDB file is similar to PDB files we have worked with before, except that it contains hydrogens.
The PQR file is another molecular file format that is similar to a PDB, but contains information about atomic radii and atomic charges.

For use with AutoDock Vina, we need our protein file to be in the "PDBQT" format. PDBQT is a specialized file format used by AutoDock Vina and other AutoDock tools. Like the PQR format, the PDBQT format can also contain partial charges. We will load our PQR file and use MDAnalysis to write a PDBQT file.

<div class="alert alert-block alert-success">
<strong>What information do we need in the PDBQT file?</strong>

We'll be using AutoDock Vina with the "vina" scoring function (this will be explained in more detail in the next notebook). The vina scoring function doesn't use charges to dock, so we could have also used the PDB file without charges to convert to a PDBQT file. However, some scoring functions do use partial charges.

</div>

In [ ]:
# make a directory for pdb files
pdbqt_directory = "pdbqt"
os.makedirs(pdbqt_directory, exist_ok=True)

u = mda.Universe(f"{protein_directory}/protein_{pdb_id}.pqr")
u.atoms.write(f"{pdbqt_directory}/{pdb_id}.pdbqt")

The PDBQT file generated by MDAnalysis includes two lines at the start of the structure that AutoDock Vina doesn't accept. 
These lines start with "TITLE" and "CRYST1". To resolve this, the following cell replaces these lines with "REMARK", which is acceptable to AutoDock Vina.

You might have also just chosen to use a different software to write your PDBQT. 
[OpenBabel](https://openbabel.org/index.html) is a popular choice. However, we are using MDAnalysis here for consistency with the rest of the workshop and to limit the number of libraries we are using.

In [ ]:
# Read in the just-written PDBQT file, replace text, and write back
with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'r') as file:
    file_content = file.read()

# Replace 'TITLE' and 'CRYST1' with 'REMARK'
file_content = file_content.replace('TITLE', 'REMARK').replace('CRYST1', 'REMARK')

# Write the modified content back to the file
with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'w') as file:
    file.write(file_content)

## Ligand Preparation

When preparing small molecule PDBQT files, you could have also chosen to use MDAnalysis or other tools.
However, we are going to use a special program for small molecules and docking called [meeko](https://github.com/forlilab/Meeko).
We choose to use meeko for our ligands because it will allow us to more easily visualize our results later.
Note that when using meeko, ligands should have hydrogens added already.

We are using the command line for meeko, similar to PDB2PQR. 
You could also choose to use the Python API for this, but the command line is simpler for common tasks like converting an SDF to a PDBQT.

In the cell below, we execute a command that converts our ligands that in we prepared in the `molecule_manipulation` notebook to a PDBQT file.

In [ ]:
# Use meeko to prepare small molecules - using meeko helps us visualize them later.
! mk_prepare_ligand.py -i ligands_to_dock/13U.sdf -o pdbqt/13U.pdbqt
! mk_prepare_ligand.py -i ligands_to_dock/13U_modified_methyl.sdf -o pdbqt/13U_modified_methyl.pdbqt
! mk_prepare_ligand.py -i ligands_to_dock/13U_modified_N.sdf -o pdbqt/13U_modified_N.pdbqt